In [1]:
import re
from konlpy.tag import Okt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import  load_model

In [2]:
import pickle
with open("data.pickle","rb") as fr:
    전처리_된_데이터 = pickle.load(fr)

In [3]:
test_inputs=전처리_된_데이터['태스트_데이터']
test_outputs=전처리_된_데이터['태스트_결과']
token=전처리_된_데이터['토큰']
w_l=전처리_된_데이터['입력길이']

In [4]:
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(test_inputs,test_outputs)[1]

930/930 [==============================] - 38s 40ms/step - loss: 0.4641 - acc: 0.7839


0.7839282155036926

In [5]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 13812), started 0:35:42 ago. (Use '!kill 13812' to kill it.)

In [6]:
def 문장_전처리(문장):
    okt=Okt()
    s_w=전처리_된_데이터['불용어']
    문장 = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', 문장)
    문장 = okt.morphs(문장, stem=True)
    문장 = [w for w in 문장 if not w in s_w] 
    encoded = token.texts_to_sequences([문장])
    pad_new = pad_sequences(encoded, maxlen = w_l) 
    score = float(loaded_model.predict(pad_new))
    if(score > 0.5):
        print(f"{score * 100:.2f}% 확률로 긍정 리뷰입니다.\n")
    else:
        print(f"{(1 - score) * 100:.2f}% 확률로 부정 리뷰입니다.\n")

In [7]:
문장_전처리('똥겜 너무하네 이건 완전 노가다')

89.09% 확률로 부정 리뷰입니다.



In [10]:
loaded_model.layers

In [18]:
len(loaded_model.layers[0].get_weights())

1

In [17]:
len(loaded_model.layers[0].get_weights()[0])

12614

In [21]:
len(loaded_model.layers[0].get_weights()[0][1])

128

In [23]:
token.word_counts

OrderedDict([('꼬맹이', 5),
             ('바드', 1),
             ('귀여운', 142),
             ('모험', 64),
             ('춤', 11),
             ('하고', 3457),
             ('노래', 171),
             ('독특하고', 17),
             ('딱', 469),
             ('그냥', 2427),
             ('저', 1123),
             ('냥', 179),
             ('할만', 790),
             ('함', 1831),
             ('영어', 548),
             ('안', 4537),
             ('어려움', 259),
             ('지금', 854),
             ('겜', 4482),
             ('해봤는데', 318),
             ('뭔', 464),
             ('게임', 25344),
             ('되질', 33),
             ('않는다', 196),
             ('서버', 1016),
             ('관리', 144),
             ('개', 2383),
             ('씹', 254),
             ('좃', 27),
             ('같이', 839),
             ('하네', 109),
             ('못', 3388),
             ('이딴', 185),
             ('걸', 1165),
             ('만원', 379),
             ('받고', 164),
             ('판다', 15),
             ('고', 2570),
             ('

In [31]:
d=token.word_index.items()

In [26]:
for w,i in token.word_counts.items():
    print(i)
    break

5


In [33]:
pr_data=sorted(d, key=lambda x : x[1])

In [34]:
for w,i in pr_data:
    print(i)
    break

1


In [35]:
dic_data=token.word_index

In [40]:
len(token.word_index)

59257

In [61]:
w_l_data=list(token.word_index.keys())

In [62]:
w_l_data.insert(0,'OOV')

In [63]:
w_l_data=w_l_data[:12614]

In [64]:
ws=loaded_model.layers[0].get_weights()[0]

In [65]:
len(ws),len(w_l_data)

(12614, 12614)

In [67]:
import io
out_v=io.open('v_data.tsv','w',encoding='utf-8')
out_w=io.open('w_data.tsv','w',encoding='utf-8')
for i, w in enumerate(w_l_data):
    if i==0:
        continue
    vec=ws[i]
    out_v.write('\t'.join([str(i) for i in vec])+'\n')
    out_w.write(w+'\n')
out_v.close()
out_w.close()